In [1]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.1985, 0.6721, 0.8308],
        [0.7061, 0.3165, 0.1583],
        [0.5987, 0.7386, 0.6720],
        [0.2903, 0.1774, 0.3367],
        [0.3581, 0.4806, 0.3304]])


In [2]:
torch.cuda.is_available()

False